# Homework 1 for CS 329P

**Authors**: xiaoxiao

**Emails**: myname@xiaoxiao.com

**Submission.** Please insert your names and emails above, save your code in this notebook, and explain what you are doing along with your findings in text cells. You can think of it as a technical report with code. Before submission, please use `Kernel -> Restart & Run All` in the Jupyter menu to verify your code is runnable and save all outputs. Afterwards, you can either upload your raw notebook (`hw1.ipynb`) or an exported PDF version to the `Homework 1` assignment in Canvas.


In this homework, we will train a house sales price predictor on the data we scraped previously. The purpose of this homework is to let you practice different techniques that you can use to preprocess raw data. Your job is to obtain the best root mean squared logarithmic error (RMSLE) on the test dataset. To make your job easy, we provide sample code to train a model to report RMSLE and a list of ideas you can explore.

**Note**: You can use either local runtimes to complete this assignment, or a hosted runtime (with GPU) on Colab. The second option generally runs faster. If using a local runtime, make sure that your Python version is less than 3.9 but at least 3.6, or you may have issues installing Autogluon. If using a runtime hosted on Colab, you can use the File Explorer pane on the left to upload the `house_sales.ftr` file. Make sure to wait until the file finishes uploading before running the next code block.

Additionally, if using a local runtime, please refer to the [AG document](https://auto.gluon.ai/stable/index.html#installation) for info on how to install autogluon.

## Prepare Data

Let's first read in the dataset we used in our [Exploratory Data Analysis (EDA)](https://c.d2l.ai/stanford-cs329p/_static/notebooks/cs329p_notebook_eda.slides.html). Note that we use the [`feather` format](https://arrow.apache.org/docs/python/feather.html), which is faster to read than CSV but uses more disk space. The file `home_sales.ftr` can be downloaded from the Assignments folder in Canvas.

Just for your information, it is generated with:

```python
data = pd.read_csv('house_sales.zip', dtype='unicode')
data.to_feather('house_sales.ftr')
```

The following code needs at least 2GB memory. If using a local runtime, please make sure your machine has enough memory.

----------


## 笑笑的作业感想

这一部分不算专注于模型，而是专注于数据清洗和预处理。包括了：如何处理数据、如何数据清洗等。老师似乎希望学生对比下数据清洗之后的效果。以下是我的代码。

注意：

1. 下面的部分在Google Colab上运行。因为可以蹭免费的GPU；
2. 请从上到下运行代码，在必要的地方有我的注释。

下面是老师给出的一些思路，我实现了其中的一部分。

## Your Solution

Please include your solution in the following section. (You are welcome to edit and delete code in previous sections).

Your goal is to train a model using the features in the original dataset that minimizes the RMSLE on the validation dataset. While the naïve model achieves an RMSLE of ~0.3, it is possible to achieve an RMSLE of less than 0.08 on the same dataset.

Here is a list of ideas you could explore:

- More features: We only selected a small set of columns to use in training. You can add more, especially the ones we examined in EDA.
- Data type conversion: Most data columns are strings; you may need to convert them into numerical values.
- Data cleaning: There are NAN and outliers sprinkled throughout the dataset. You should find ways to selectively filter and remove them.
- More examples: We only included sales made in 2021; there is a large number of examples in previous years that you can also include.

In [1]:
!pip install numpy pandas autogluon mxnet --upgrade
# 安装需要的依赖。

  Using cached numpy-2.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)


In [3]:
# Run the following line once to install. You may need to restart your runtime afterwards:
# !pip3 install numpy pandas autogluon mxnet --upgrade
import pandas as pd
import numpy as np
import pyarrow
print(pyarrow.__version__)
data = pd.read_feather('./house_sales2.ftr')

17.0.0


In [4]:
data.describe()

,Id,Address,Sold Price,Sold On,Summary,Type,Year built,Heating,Cooling,Parking,...,Well Disclosure,remodeled,DOH2,SerialX,Full Baths,Tax Legal Lot Number,Tax Legal Block Number,Tax Legal Tract Number,Building Name,Zip
count,164944,164944,164859,164944,161827,163260,163289,163266,163263,163260,...,1,1,1,1,1,1,1,1,1,164944
unique,164944,161952,11784,1101,159689,317,182,3284,1044,12214,...,1,1,1,1,1,1,1,1,1,1762
top,2080183300,"Zzzz,","$1,200,000",02/26/21,For comp purposes only.,SingleFamily,No Data,No Data,No Data,"Garage, Garage - Attached, Covered",...,Yes,2020.0,TBD,0903-521-14085B,One,39.0,62033.0,Piru-0057,Iron Horse South,95003
freq,1,97,1149,1979,46,102040,14224,37341,52958,28165,...,1,1,1,1,1,1,1,1,1,795


In [5]:
import scipy
import numpy as np
scipy.__version__, np.__version__

('1.12.0', '1.26.4')

We select a few common columns to make our training fast. You need to select more columns to make your model more accurate.

In [18]:
null_sum = data.isnull().sum()
# data.columns[null_sum < len(data) * 0.3]

data.drop(columns=data.columns[null_sum > len(data) * 0.3], inplace=True)
data.columns
# 看一下NULL值占比大于30%的那些列，原地替换

# df = data[['Sold Price', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Bathrooms']].copy()
# uncomment the below line to save memory
# del data

Index(['Id', 'Address', 'Sold Price', 'Sold On', 'Summary', 'Type',
       'Year built', 'Heating', 'Cooling', 'Parking', 'Bedrooms', 'Bathrooms',
       'Total interior livable area', 'Total spaces', 'Garage spaces',
       'Home type', 'Region', 'Elementary School', 'Elementary School Score',
       'Elementary School Distance', 'High School', 'High School Score',
       'High School Distance', 'Heating features', 'Parking features',
       'Lot size', 'Parcel number', 'Tax assessed value', 'Annual tax amount',
       'Listed On', 'Listed Price', 'Zip'],
      dtype='object')

In [19]:
data.dtypes

,0
Id,object
Address,object
Sold Price,object
Sold On,object
Summary,object
Type,object
Year built,object
Heating,object
Cooling,object
Parking,object


In [20]:
currency = ['Sold Price', 'Listed Price', 'Tax assessed value', 'Annual tax amount']

for c in currency:
    data[c] = data[c].replace(
          r'[$,-]', '', regex=True).replace(
              r'^\s*$', np.nan, regex=True).astype(float)
# 用正则表达式删除那些货币符号，把数据转换成数字

In [22]:
areas = ["Total interior livable area", 'Lot size']
for c in areas:
    acres = data[c].str.contains('Acres') == True
    col = data[c].replace(r'\b sqft\b|\b Acres\b|\b,\b', '',regex = True).astype(float)
    col[acres] *= 43560
    data[c] = col
# 用正则表达式删除面积的单位，统一数据

In [23]:
abnormal = (data[areas[1]] < 10) | (data[areas[1]] > 1e4)
data = data[-abnormal]
sum(abnormal)
# 查看异常数据

41000

We copy the code from EDA to convert `Sold Price` to numerical values, which is our prediction target. We also remove examples whose prices are too high or too low.

In [26]:
df = data[['Sold Price', 'Listed Price', 'Annual tax amount', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Bathrooms', "Total interior livable area", 'Lot size']].copy()
c = 'Sold Price'
print(f"Before: {df.shape}")
df['Sold Price'] = np.log10(df['Sold Price'])
# 先把一部分数据转换成数字，
# 把SOLD PRICE数据取了log，这样消除了数据大于10^8和数据小于 10^4  的房子
df = df[(df['Sold Price'] >= 4 ) & (df['Sold Price'] <= 8 )]
print(f"After: {df.shape}")




Before: (123944, 10)
After: (120754, 10)


We use the house sales between 2021-2-15 and 2021-3-1 as our test data. You can use any example before 2021-2-15, but not after. In other words, we pretend we are launching our model on 2021-2-15 and testing it for 2 weeks. Here we only use sales in 2021 for fast training, but you can use more to improve accuracy.

- 把2021-2-15～2021-3-1 的数据作为测试集。我们可以用在此之前的数据作为训练集进行测试，但是不能用在此之后的。
- 初始版本里用的是2021年2-15之前的作为训练集；我的版本用了2020-06-01～2021-02-14的数据进行训练


In [28]:
test_start, test_end = pd.Timestamp(2021, 2, 15), pd.Timestamp(2021, 3, 1)
train_start = pd.Timestamp(2020, 6, 1)
df['Sold On'] = pd.to_datetime(df['Sold On'], errors='coerce')
train = df[(df['Sold On'] >= train_start) & (df['Sold On'] < test_start)]
test = df[(df['Sold On'] >= test_start) & (df['Sold On'] < test_end)]
# 截取数据
print(train.shape, test.shape)

(53468, 10) (8339, 10)


Define our evaluation metric.

In [30]:
def rmsle(y_hat, y):
    # we already used log prices before, so we only need to compute RMSE
    return sum((y_hat - y)**2 / len(y))**0.5

## AutoGluon Baseline

We provide a baseline model trained by AutoGluon (AG). AG is an automl tool that performs automatic feature engineering, model selections, and ensemble. You are welcome to use any model and tool in achieving the best results possible in your homework. However, we recommend that you reuse the following training code so that you can focus on data preprocessing.

In [31]:
from autogluon.tabular import TabularPredictor

label = 'Sold Price'
predictor = TabularPredictor(label=label).fit(train)

No path specified. Models will be saved in: "AutogluonModels/ag-20240922_042750"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       9.90 GB / 12.67 GB (78.1%)
Disk Space Avail:   69.29 GB / 112.64 GB (61.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fa

[1000]	valid_set's rmse: 0.218715


	-0.2186	 = Validation score   (-root_mean_squared_error)
	5.1s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: LightGBM ...
	-0.22	 = Validation score   (-root_mean_squared_error)
	3.48s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-0.2325	 = Validation score   (-root_mean_squared_error)
	179.9s	 = Training   runtime
	0.32s	 = Validation runtime
Fitting model: CatBoost ...
	-0.2222	 = Validation score   (-root_mean_squared_error)
	108.58s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-0.2304	 = Validation score   (-root_mean_squared_error)
	66.88s	 = Training   runtime
	0.5s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-0.2493	 = Validation score   (-root_mean_squared_error)
	50.74s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: XGBoost ...
	-0.2214	 = Validation score   (-root_mean_squared_error)
	5.1s	 = Training   runtime
	0.03s	 = Validation runtime
Fi

Test the performance of each model.

In [32]:
predictor.leaderboard(test, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.105996,-0.217586,root_mean_squared_error,1.537560,0.693659,132.287524,0.004228,0.000529,0.024418,2,True,12
1,LightGBMXT,-0.106400,-0.218624,root_mean_squared_error,0.528932,0.120913,5.102951,0.528932,0.120913,5.102951,1,True,3
2,CatBoost,-0.107040,-0.222217,root_mean_squared_error,0.067002,0.021124,108.584581,0.067002,0.021124,108.584581,1,True,6
3,ExtraTreesMSE,-0.107070,-0.230382,root_mean_squared_error,0.746778,0.497813,66.883033,0.746778,0.497813,66.883033,1,True,7
4,LightGBM,-0.107083,-0.219956,root_mean_squared_error,0.100228,0.032041,3.478999,0.100228,0.032041,3.478999,1,True,4
5,XGBoost,-0.110969,-0.221414,root_mean_squared_error,0.094999,0.031327,5.099067,0.094999,0.031327,5.099067,1,True,9
6,RandomForestMSE,-0.112785,-0.232487,root_mean_squared_error,1.397590,0.321162,179.900616,1.397590,0.321162,179.900616,1,True,5
7,NeuralNetTorch,-0.119217,-0.230462,root_mean_squared_error,0.064962,0.032017,56.707263,0.064962,0.032017,56.707263,1,True,10
8,LightGBMLarge,-0.122203,-0.220121,root_mean_squared_error,0.192661,0.042387,3.569859,0.192661,0.042387,3.569859,1,True,11
9,NeuralNetFastAI,-0.138768,-0.249298,root_mean_squared_error,0.223903,0.041143,50.737153,0.223903,0.041143,50.737153,1,True,8


- 对比初始版本的结果：

![](https://cdn.jsdelivr.net/gh/SmilingWayne/picsrepo/202409221507945.png)

Next, we compute the importance of each feature, along with several other metrics. It loooks like the `Sold On` feature is not very useful, likely because the houses in the test data were all sold late. You can choose to either remove such a feature, or find a way to extract a more useful presentation from it.

In [33]:
predictor.feature_importance(test)

Computing feature importance via permutation shuffling for 9 features using 5000 rows with 5 shuffle sets...
	124.88s	= Expected runtime (24.98s per shuffle set)
	51.05s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Listed Price,0.321596,0.003408,1.513566e-09,5,0.328613,0.314578
Annual tax amount,0.066687,0.004223,1.920186e-06,5,0.075383,0.057991
Type,0.026792,0.002577,1.015192e-05,5,0.032098,0.021485
Year built,0.009870,0.001137,2.076148e-05,5,0.012211,0.007529
Total interior livable area,0.007420,0.001202,7.995196e-05,5,0.009896,0.004944
Bathrooms,0.003026,0.000667,2.665182e-04,5,0.004400,0.001652
Bedrooms,0.002889,0.000605,2.185214e-04,5,0.004136,0.001642
Lot size,0.002498,0.000711,7.071855e-04,5,0.003961,0.001035
Sold On,0.000431,0.000378,3.177144e-02,5,0.001210,-0.000348


Finally, let's predict and evaluate the RMSLE.

In [34]:
preds = predictor.predict(test.drop(columns=[label]))
rmsle(preds, test[label])
# 初始版本的误差：0.28302552009742815

0.10599613970306478